In [1]:
from pathlib import Path

import pandas as pd

In [8]:
data_dir = Path.cwd().parent / "data"
strategies_dir = data_dir / "strategies"
returns_dir = data_dir / "returns"

### Load data

In [3]:
df_weights = pd.read_csv(strategies_dir / "V_W.csv")

In [4]:
df_crsp = pd.read_csv(data_dir / "crsp_raw.csv", parse_dates=["date"])
df_crsp["date"] = df_crsp["date"].dt.to_period("M")

# Filter for companies in df_weights
valid_permnos = df_weights["PERMNO"].unique()
df_crsp = df_crsp[df_crsp["PERMNO"].isin(valid_permnos)]

# Drop columns
df_crsp = df_crsp.drop(columns=["SHRCD", "EXCHCD"], errors="ignore")

# Drop non-numeric RET
df_crsp["RET"] = pd.to_numeric(df_crsp["RET"], errors="coerce")
df_crsp = df_crsp.dropna(subset=["RET"])

# # Remove duplicates, keeping the first occurrence
# df_crsp = df_crsp.drop_duplicates(subset=["PERMNO", "date"])

df_crsp.to_csv(data_dir / "monthly_returns.csv", index=False)

### Compute returns

In [5]:
def calculate_strategy_returns(
    df_weights, df_returns, years_shift, n_month_rebalance, cols
):
    # Prepare returns
    df_ret = df_returns.copy()
    # Ensure date is datetime or period, extract year
    df_ret["year"] = df_ret["date"].dt.year

    # Calculate the year of the weights we want to match with
    df_ret["match_year"] = df_ret["year"] + years_shift

    # Prepare weights
    # df_weights should have 'date' as year integer
    df_w = df_weights.copy()
    # Rename date to match_year for merging
    df_w = df_w.rename(columns={"date": "match_year"})

    # Merge returns with weights
    # We keep all returns, and attach weights where available
    df_merged = pd.merge(df_ret, df_w, on=["PERMNO", "match_year"], how="left")

    # Sort by date to ensure chronological order
    df_merged = df_merged.sort_values("date")

    unique_dates = df_merged["date"].unique()
    results = {col: [] for col in cols}

    # Pre-group by date for performance
    grouped = df_merged.groupby("date")

    for col in cols:
        current_value = 1.0
        # Holdings: Series of $ value per PERMNO
        current_holdings = pd.Series(dtype=float)

        col_returns = []

        for i, date in enumerate(unique_dates):
            # Get data for this month
            try:
                month_data = grouped.get_group(date)
            except KeyError:
                col_returns.append(0.0)
                continue

            # Rebalance logic
            # Rebalance at the start (i==0) or every n months
            if i % n_month_rebalance == 0:
                # Get target weights for this month (based on the matched year)
                valid_weights = month_data.dropna(subset=[col])

                if not valid_weights.empty:
                    # Normalize weights to invest fully in available assets
                    w = valid_weights.set_index("PERMNO")[col]
                    w_sum = w.sum()
                    if w_sum != 0:
                        w = w / w_sum
                        current_holdings = w * current_value
                    else:
                        # Weights sum to 0? Keep cash or previous holdings?
                        # Assuming cash if weights are invalid
                        current_holdings = pd.Series(dtype=float)
                else:
                    # No weights available for this period
                    # If it's the start, we can't invest.
                    if i == 0:
                        current_holdings = pd.Series(dtype=float)
                    # Else keep previous holdings (drift)

            # Calculate Returns for this month
            if current_holdings.empty:
                col_returns.append(0.0)
                continue

            # Get returns vector for this month
            # We assume missing returns (delisting etc) are 0.0 or handled in data
            rets = month_data.set_index("PERMNO")["RET"]

            # Align holdings with returns
            # Use reindex to keep only held stocks, fill missing returns with 0
            aligned_rets = rets.reindex(current_holdings.index, fill_value=0.0)

            # Update holdings value based on returns
            new_holdings = current_holdings * (1 + aligned_rets)

            new_value = new_holdings.sum()

            if current_value != 0:
                ret = new_value / current_value - 1
            else:
                ret = 0.0

            col_returns.append(ret)

            # Update state
            current_value = new_value
            current_holdings = new_holdings

        results[col] = col_returns

    df_res = pd.DataFrame(results)
    df_res["date"] = unique_dates
    return df_res

In [6]:
ID_COLS = ["PERMNO", "date", "tic", "conm"]
factor_combs = [["V", "W"]]
suffixes = [
    "_int_ter",
    "_mix_ter",
    "_int_dec",
    "_mix_dec",
    "_int_bw",
    "_mix_bw",
]
monthly_returns = df_crsp


In [10]:
for factor_comb in factor_combs:
    factor_str = "_".join(factor_comb)
    weights_cols = [factor_str + s for s in suffixes]

    df_r = calculate_strategy_returns(df_weights, monthly_returns, 1, 1, weights_cols)

    df_r.to_csv(returns_dir / f"{factor_str}.csv", index=False)